In [43]:
from libpgm.nodedata import NodeData
from libpgm.graphskeleton import GraphSkeleton
from libpgm.discretebayesiannetwork import DiscreteBayesianNetwork
from libpgm.lgbayesiannetwork import LGBayesianNetwork
from libpgm.hybayesiannetwork import HyBayesianNetwork
from libpgm.dyndiscbayesiannetwork import DynDiscBayesianNetwork
from libpgm.tablecpdfactorization import TableCPDFactorization
from libpgm.sampleaggregator import SampleAggregator
from libpgm.pgmlearner import PGMLearner

import numpy as np
#create table
from prettytable import PrettyTable

#read csv
import pandas as pd

In [44]:
data=pd.read_csv("./Training/first_step_prepro/training_initial_sentimental.csv",sep='[;,|]', engine='python')
data.query('overall > 0',inplace=True)
data.query('clean > 0',inplace=True)
data.query('value >0',inplace=True)
data.query('room >0',inplace=True)
data.query('location >0',inplace=True)
data.query('check >0',inplace=True)
data.query('service >0',inplace=True)
data.query('business >0',inplace=True)
data.query('target != "neutro"',inplace=True)
data['id'] = np.arange(len(data))

cols=data.columns.tolist()
cols= cols[-1:]+cols[:-1]
data=data[cols]
data.to_csv("./csv/training_sentimental.csv",index=False)
all_data=len(data)
data.head(5)

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
0,0,hotel_231505,-1,-1,5,5,5,5,5,5,5,5,good
1,1,hotel_231505,-1,-1,1,1,1,3,2,1,3,3,good
2,2,hotel_231505,-1,-1,3,3,2,3,4,2,2,3,good
5,3,hotel_231505,4,4,4,4,4,4,4,4,3,5,good
7,4,hotel_231505,3,3,4,4,4,3,4,5,4,3,good


In [45]:
#data set dopo il processo di undersampling con weka
data=pd.read_csv("./csv/training_sentimental_undersampling.csv")
del data['id']
data['id'] = np.arange(len(data))
cols=data.columns.tolist()
cols= cols[-1:]+cols[:-1]
data=data[cols]
all_data=len(data)
data.head()

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
0,0,hotel_305810,2,1,3,3,3,3,3,3,3,3,good
1,1,hotel_572859,10,10,5,5,5,5,5,5,5,5,good
2,2,hotel_230875,-1,-1,3,3,3,5,5,3,3,4,good
3,3,hotel_234752,1,0,5,5,3,5,5,4,4,3,good
4,4,hotel_228621,-1,-1,5,5,5,4,5,3,4,3,good


In [46]:
bad=data.query('target == "bad" and overall >0')
good=data.query('target == "good"')
bad_n=len(bad)
good_n=len(good)

print(float(good_n)/all_data,float(bad_n)/all_data)

(0.5651997291807719, 0.4348002708192282)


In [47]:
#numero di volte per ogni valore di clean dato good
clean_g_1=data.query('target == "good" and clean == 1')
clean_g_2=data.query('target == "good" and clean == 2')
clean_g_3=data.query('target == "good" and clean == 3')
clean_g_4=data.query('target == "good" and clean == 4')
clean_g_5=data.query('target == "good" and clean == 5')
clean_g=[len(clean_g_1),len(clean_g_2),len(clean_g_3),len(clean_g_4),len(clean_g_5)]
clean_g

#numero di volte per ogni valore di clean dato bad
clean_b_1=data.query('target == "bad" and clean == 1')
clean_b_2=data.query('target == "bad" and clean == 2')
clean_b_3=data.query('target == "bad" and clean == 3')
clean_b_4=data.query('target == "bad" and clean == 4')
clean_b_5=data.query('target == "bad" and clean == 5')
clean_b=[len(clean_b_1),len(clean_b_2),len(clean_b_3),len(clean_b_4),len(clean_b_5)]
clean_b

#numero di volte per ogni valore di value dato good
value_g_1=data.query('target == "good" and value == 1')
value_g_2=data.query('target == "good" and value == 2')
value_g_3=data.query('target == "good" and value == 3')
value_g_4=data.query('target == "good" and value == 4')
value_g_5=data.query('target == "good" and value == 5')
value_g=[len(value_g_1),len(value_g_2),len(value_g_3),len(value_g_4),len(value_g_5)]

#numero di volte per ogni valore di value dato bad
value_b_1=data.query('target == "bad" and value == 1')
value_b_2=data.query('target == "bad" and value == 2')
value_b_3=data.query('target == "bad" and value == 3')
value_b_4=data.query('target == "bad" and value == 4')
value_b_5=data.query('target == "bad" and value == 5')
value_b=[len(value_b_1),len(value_b_2),len(value_b_3),len(value_b_4),len(value_b_5)]

#numero di volte per ogni valore di room dato good
room_g_1=data.query('target == "good" and room == 1')
room_g_2=data.query('target == "good" and room == 2')
room_g_3=data.query('target == "good" and room == 3')
room_g_4=data.query('target == "good" and room == 4')
room_g_5=data.query('target == "good" and room == 5')
room_g=[len(room_g_1),len(room_g_2),len(room_g_3),len(room_g_4),len(room_g_5)]

#numero di volte per ogni valore di room dato bad
room_b_1=data.query('target == "bad" and room == 1')
room_b_2=data.query('target == "bad" and room == 2')
room_b_3=data.query('target == "bad" and room == 3')
room_b_4=data.query('target == "bad" and room == 4')
room_b_5=data.query('target == "bad" and room == 5')
room_b=[len(room_b_1),len(room_b_2),len(room_b_3),len(room_b_4),len(room_b_5)]

#numero di volte per ogni valore di location dato good
loc_g_1=data.query('target == "good" and location == 1')
loc_g_2=data.query('target == "good" and location == 2')
loc_g_3=data.query('target == "good" and location == 3')
loc_g_4=data.query('target == "good" and location == 4')
loc_g_5=data.query('target == "good" and location == 5')
loc_g=[len(loc_g_1),len(loc_g_2),len(loc_g_3),len(loc_g_4),len(loc_g_5)]

#numero di volte per ogni valore di location dato bad
loc_b_1=data.query('target == "bad" and location == 1')
loc_b_2=data.query('target == "bad" and location == 2')
loc_b_3=data.query('target == "bad" and location == 3')
loc_b_4=data.query('target == "bad" and location == 4')
loc_b_5=data.query('target == "bad" and location == 5')
loc_b=[len(loc_b_1),len(loc_b_2),len(loc_b_3),len(loc_b_4),len(loc_b_5)]

#numero di volte per ogni valore di check dato good
check_g_1=data.query('target == "good" and check == 1')
check_g_2=data.query('target == "good" and check == 2')
check_g_3=data.query('target == "good" and check == 3')
check_g_4=data.query('target == "good" and check == 4')
check_g_5=data.query('target == "good" and check == 5')
check_g=[len(check_g_1),len(check_g_2),len(check_g_3),len(check_g_4),len(check_g_5)]

#numero di volte per ogni valore di check dato bad
check_b_1=data.query('target == "bad" and check == 1')
check_b_2=data.query('target == "bad" and check == 2')
check_b_3=data.query('target == "bad" and check == 3')
check_b_4=data.query('target == "bad" and check == 4')
check_b_5=data.query('target == "bad" and check == 5')
check_b=[len(check_b_1),len(check_b_2),len(check_b_3),len(check_b_4),len(check_b_5)]

#numero di volte per ogni valore di service dato good
ser_g_1=data.query('target == "good" and service == 1')
ser_g_2=data.query('target == "good" and service == 2')
ser_g_3=data.query('target == "good" and service == 3')
ser_g_4=data.query('target == "good" and service == 4')
ser_g_5=data.query('target == "good" and service == 5')
ser_g=[len(ser_g_1),len(ser_g_2),len(ser_g_3),len(ser_g_4),len(ser_g_5)]

#numero di volte per ogni valore di service dato bad
ser_b_1=data.query('target == "bad" and service == 1')
ser_b_2=data.query('target == "bad" and service == 2')
ser_b_3=data.query('target == "bad" and service == 3')
ser_b_4=data.query('target == "bad" and service == 4')
ser_b_5=data.query('target == "bad" and service == 5')
ser_b=[len(ser_b_1),len(ser_b_2),len(ser_b_3),len(ser_b_4),len(ser_b_5)]

#numero di volte per ogni valore di business dato good
b_g_1=data.query('target == "good" and business == 1')
b_g_2=data.query('target == "good" and business == 2')
b_g_3=data.query('target == "good" and business == 3')
b_g_4=data.query('target == "good" and business == 4')
b_g_5=data.query('target == "good" and business == 5')
b_g=[len(b_g_1),len(b_g_2),len(b_g_3),len(b_g_4),len(b_g_5)]

#numero di volte per ogni valore di service dato bad
b_b_1=data.query('target == "bad" and business == 1')
b_b_2=data.query('target == "bad" and business == 2')
b_b_3=data.query('target == "bad" and business == 3')
b_b_4=data.query('target == "bad" and business == 4')
b_b_5=data.query('target == "bad" and business == 5')
b_b=[len(b_b_1),len(b_b_2),len(b_b_3),len(b_b_4),len(b_b_5)]

#create view to cpt
def generate_cpt(name,array_g,array_b):
    cpt= PrettyTable()
    cpt.field_names =["P("+name+"|target)",1,2,3,4,5]
    good_c=['good']
    bad_c=['bad']
    for i in range(len(array_g)):
        good_c.append(float(array_g[i])/good_n)
        bad_c.append(float(array_b[i])/bad_n)
    cpt.add_row(good_c)
    cpt.add_row(bad_c)
    return cpt

print(generate_cpt("clean",clean_g,clean_b))
print(generate_cpt("value",value_g,value_b))
print(generate_cpt("room",room_g,room_b))
print(generate_cpt("location",loc_g,loc_b))
print(generate_cpt("check",check_g,check_b))
print(generate_cpt("service",ser_g,ser_b))
print(generate_cpt("business",b_g,b_b))

+-----------------+-----------------+-----------------+-----------------+----------------+----------------+
| P(clean|target) |        1        |        2        |        3        |       4        |       5        |
+-----------------+-----------------+-----------------+-----------------+----------------+----------------+
|       good      | 0.0148538572113 | 0.0268327743172 | 0.0836128413991 | 0.236942980355 | 0.637757546718 |
|       bad       |  0.275615073186 |  0.201183431953 |  0.232326378075 | 0.161009031454 | 0.129866085332 |
+-----------------+-----------------+-----------------+-----------------+----------------+----------------+
+-----------------+-----------------+-----------------+----------------+----------------+-----------------+
| P(value|target) |        1        |        2        |       3        |       4        |        5        |
+-----------------+-----------------+-----------------+----------------+----------------+-----------------+
|       good      | 0.025874

In [48]:
#testing
data=pd.read_csv("./Testing/first_step_prepro/testing_initial_sentimental.csv",sep='[;,|]', engine='python')
data.query('overall > 0',inplace=True)
data.query('clean > 0',inplace=True)
data.query('value >0',inplace=True)
data.query('room >0',inplace=True)
data.query('location >0',inplace=True)
data.query('check >0',inplace=True)
data.query('service >0',inplace=True)
data.query('business >0',inplace=True)
data.query('target != "neutro"',inplace=True)
data['id'] = np.arange(len(data))

cols=data.columns.tolist()
cols= cols[-1:]+cols[:-1]
data=data[cols]
data.to_csv("./csv/testing_sentimental.csv",index=False)
all_data=len(data)
data.head(5)

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
2,0,hotel_80799,-1,-1,3,5,3,3,3,5,3,4,good
5,1,hotel_80799,1,1,1,1,1,1,1,3,2,1,good
6,2,hotel_80799,2,1,4,5,3,4,5,5,5,3,good
9,3,hotel_80799,-1,-1,1,1,1,1,3,2,3,2,bad
13,4,hotel_80799,3,3,1,2,1,3,2,5,5,1,bad


In [49]:
#inferenza con naive bayes con sentimental analysis
skel= GraphSkeleton()
nd =NodeData()

skel.load('./skeleton_sentimental.txt')
nd.load("./skeleton_sentimental.txt")

skel.toporder()
bn=DiscreteBayesianNetwork(skel, nd)

def inference(row):
    fn =TableCPDFactorization(bn)
    evidence =dict(clean=row[8],value=row[5],room=row[6],location=row[7],check=row[9],service=row[10],business=row[11])
    query= {'target' :['bad']}
    bad = fn.specificquery(query,evidence)
    good=1-bad
    return (good,bad)

list_result=[]
list_good_sent=[]
for i in range(len(data)):
    row=data.query('id=='+str(i))
    result_row=row.values.tolist()[0]
    good,bad=inference(result_row)
    list_good_sent.append(good)
    print(good,bad)
    if good < bad:
        list_result.append('bad')
    else:
        list_result.append('good')
data['target']=list_result
        

(0.8560259127606903, 0.1439740872393097)
(1.7849347733189802e-07, 0.9999998215065227)
(0.9990035145095844, 0.0009964854904156775)
(3.292860474735093e-06, 0.9999967071395253)
(0.0031857259906186153, 0.9968142740093814)
(2.0190274552245313e-07, 0.9999997980972545)
(5.988243424459583e-07, 0.9999994011756576)
(0.9999942028610157, 5.797138984308302e-06)
(0.9382862652206676, 0.06171373477933237)
(0.9972022164167559, 0.0027977835832441707)
(0.932510911985936, 0.06748908801406395)
(0.10900588274388512, 0.8909941172561149)
(0.0034809101559385436, 0.9965190898440615)
(0.0024285885886540237, 0.997571411411346)
(0.06110278872619701, 0.938897211273803)
(2.221365952292942e-07, 0.9999997778634048)
(6.741044034086485e-08, 0.9999999325895597)
(6.729993118348432e-09, 0.9999999932700069)
(0.9224378351673368, 0.07756216483266318)
(6.729993118348432e-09, 0.9999999932700069)
(0.019033047717880502, 0.9809669522821195)
(2.084568404225884e-06, 0.9999979154315958)
(5.696987503966966e-08, 0.999999943030125)
(0.9

(0.9999770153365785, 2.2984663421534405e-05)
(0.9998910497115685, 0.00010895028843153037)
(0.9999942028610157, 5.797138984308302e-06)
(0.9995956208209803, 0.0004043791790197107)
(0.98764816755815, 0.012351832441850041)
(0.9999760371169619, 2.396288303803522e-05)
(0.999990101355608, 9.898644392071408e-06)
(0.9999632631128141, 3.673688718589553e-05)
(0.006485892492447687, 0.9935141075075523)
(0.9999942028610157, 5.797138984308302e-06)
(0.999990101355608, 9.898644392071408e-06)
(0.999990101355608, 9.898644392071408e-06)
(0.9466429707873238, 0.053357029212676196)
(2.712881896904662e-08, 0.999999972871181)
(0.9999865389637633, 1.3461036236626867e-05)
(0.0026190917978166084, 0.9973809082021834)
(0.9999942028610157, 5.797138984308302e-06)
(2.4201352288089062e-05, 0.9999757986477119)
(0.9999621387615909, 3.786123840912384e-05)
(0.9999942028610157, 5.797138984308302e-06)
(0.9997810416469409, 0.00021895835305905378)
(0.9999343761249075, 6.562387509240816e-05)
(0.9999942028610157, 5.7971389843083

(0.6793411542264196, 0.32065884577358034)
(0.9999477647144017, 5.223528559830012e-05)
(0.9998508703045885, 0.00014912969541142936)
(0.9999469752104114, 5.3024789588531906e-05)
(0.9999568769846168, 4.312301538320226e-05)
(0.9894501675961702, 0.010549832403829779)
(0.033017094584367745, 0.9669829054156323)
(0.859644187789095, 0.14035581221090498)
(0.999990101355608, 9.898644392071408e-06)
(0.9997501549947179, 0.0002498450052821057)
(0.0018489327530080146, 0.998151067246992)
(0.9999942028610157, 5.797138984308302e-06)
(0.35081604105244046, 0.6491839589475595)
(0.9017754745423672, 0.09822452545763288)
(0.999981428299345, 1.857170065500709e-05)
(0.9998912641347336, 0.0001087358652663574)
(0.999990101355608, 9.898644392071408e-06)
(0.9999497219796583, 5.0278020341702656e-05)
(0.9999942028610157, 5.797138984308302e-06)
(0.9999942028610157, 5.797138984308302e-06)
(0.9999942028610157, 5.797138984308302e-06)
(0.9999443591931465, 5.564080685353149e-05)
(1.7484041586302013e-05, 0.9999825159584137)

(0.9999942028610157, 5.797138984308302e-06)
(0.8396218880547324, 0.16037811194526758)
(0.9999942028610157, 5.797138984308302e-06)
(5.1771649499654515e-06, 0.99999482283505)
(0.9970421071153368, 0.002957892884663195)
(0.999720123234209, 0.0002798767657910469)
(0.9662597773169215, 0.03374022268307856)
(0.5752563425829942, 0.4247436574170058)
(0.997086022140131, 0.0029139778598690326)
(0.6443624772223068, 0.3556375227776932)
(0.0001233536318905326, 0.9998766463681095)
(0.9999859660608581, 1.4033939141906462e-05)
(0.5267111029443032, 0.4732888970556968)
(0.20980334828099223, 0.7901966517190078)
(0.9999942028610157, 5.797138984308302e-06)
(0.9063438098532816, 0.09365619014671847)
(0.9998411486782619, 0.0001588513217380756)
(0.999905692213404, 9.430778659597239e-05)
(0.0065815434794714855, 0.9934184565205285)
(0.9999605274248923, 3.9472575107750374e-05)
(0.26776507715165343, 0.7322349228483466)
(0.31848629340821966, 0.6815137065917803)
(0.9130592863401548, 0.08694071365984524)
(1.26949705403

(0.8492279596442851, 0.15077204035571495)
(0.741626142073906, 0.258373857926094)
(0.9999942028610157, 5.797138984308302e-06)
(0.9998735551354779, 0.0001264448645220593)
(0.999981428299345, 1.857170065500709e-05)
(0.8919340523665927, 0.10806594763340724)
(0.9980888125333065, 0.0019111874666934944)
(0.9995635661087515, 0.0004364338912485189)
(0.9522953046673686, 0.047704695332631386)
(0.999981428299345, 1.857170065500709e-05)
(0.9998823178191031, 0.0001176821808968946)
(0.9896284375651403, 0.010371562434859673)
(0.9997717247563844, 0.0002282752436155975)
(1.9678323536997944e-05, 0.999980321676463)
(0.5061972464673938, 0.49380275353260616)
(0.9996517330405643, 0.0003482669594356377)
(0.999981428299345, 1.857170065500709e-05)
(0.9989441938692514, 0.0010558061307486663)
(0.9999682890129885, 3.171098701149267e-05)
(0.9998823178191031, 0.0001176821808968946)
(2.1906466114995915e-05, 0.999978093533885)
(0.9999942028610157, 5.797138984308302e-06)
(0.9997050425612034, 0.00029495743879654043)
(0.

...

(0.999823586263902, 0.00017641373609796784)
(0.9961113977031525, 0.0038886022968474786)
(0.9636151142502438, 0.03638488574975623)
(0.9787387154560805, 0.0212612845439195)
(0.9996128842620529, 0.0003871157379471531)
(0.8500190898774903, 0.1499809101225097)
(0.999896676203996, 0.00010332379600401025)
(0.8952512348188817, 0.10474876518111825)
(0.8842670605909247, 0.11573293940907528)
(0.9959004891567378, 0.004099510843262207)
(0.9999942028610157, 5.797138984308302e-06)
(0.999990101355608, 9.898644392071408e-06)
(6.729993118348432e-09, 0.9999999932700069)
(0.999981428299345, 1.857170065500709e-05)
(0.9999942028610157, 5.797138984308302e-06)
(0.999856109672115, 0.00014389032788501686)
(2.6658863823580958e-05, 0.9999733411361764)
(0.7655868188955559, 0.23441318110444412)
(0.9999859660608581, 1.4033939141906462e-05)
(0.7737985466629003, 0.2262014533370997)
(0.9999942028610157, 5.797138984308302e-06)
(0.9999859660608581, 1.4033939141906462e-05)
(0.9999942028610157, 5.797138984308302e-06)
(0.06

(0.9832932536347796, 0.016706746365220446)
(0.3181063911684884, 0.6818936088315116)
(0.9997020042238062, 0.000297995776193848)
(0.04074605677579746, 0.9592539432242025)
(0.9999865389637633, 1.3461036236626867e-05)
(0.06427212444789177, 0.9357278755521082)
(0.9977189749245883, 0.0022810250754116738)
(0.24880885136287711, 0.7511911486371229)
(0.2909754835578484, 0.7090245164421516)
(0.9991274103134039, 0.0008725896865960879)
(0.9980304326586288, 0.0019695673413712366)
(0.9999205096042536, 7.949039574641083e-05)
(0.9817335552844675, 0.018266444715532473)
(0.9998217688197736, 0.00017823118022639668)
(0.9999942028610157, 5.797138984308302e-06)
(0.06427212444789177, 0.9357278755521082)
(0.001337959221682672, 0.9986620407783173)
(0.9927603378558426, 0.007239662144157335)
(0.2228721566641011, 0.7771278433358989)
(0.6981212451623557, 0.30187875483764426)
(6.729993118348432e-09, 0.9999999932700069)
(0.9833505687563631, 0.016649431243636947)
(0.9961390826904722, 0.0038609173095277938)
(0.99971743

(0.9999534451241867, 4.655487581334263e-05)
(0.9999522688862668, 4.773111373326793e-05)
(0.0030019923334287935, 0.9969980076665712)
(0.9998075867105489, 0.0001924132894511154)
(0.032884681648828074, 0.9671153183511719)
(0.9999942028610157, 5.797138984308302e-06)
(0.9999859660608581, 1.4033939141906462e-05)
(0.9999522688862668, 4.773111373326793e-05)
(0.9999942028610157, 5.797138984308302e-06)
(0.9999942028610157, 5.797138984308302e-06)
(0.999426519411162, 0.0005734805888380405)
(0.9999942028610157, 5.797138984308302e-06)
(0.9999632631128141, 3.673688718589553e-05)
(0.9958243751476239, 0.004175624852376092)
(0.9999770153365785, 2.2984663421534405e-05)
(0.9959004891567378, 0.004099510843262207)
(0.978228591402794, 0.021771408597206082)
(0.9707869187548344, 0.029213081245165605)
(1.1148750522771422e-07, 0.9999998885124948)
(0.999004299298136, 0.000995700701864012)
(0.9983984020005132, 0.0016015979994868095)
(0.00090776108462165, 0.9990922389153784)
(0.9918094423802964, 0.00819055761970354

(0.999799072981039, 0.00020092701896106505)
(0.9996552837615601, 0.0003447162384399238)
(0.9997929245474674, 0.00020707545253262168)
(0.00032801122356984624, 0.9996719887764302)
(0.9999942028610157, 5.797138984308302e-06)
(0.739721082415882, 0.2602789175841181)
(0.9672502313541, 0.03274976864589994)
(5.4223182977874984e-05, 0.9999457768170221)
(0.9999942028610157, 5.797138984308302e-06)
(0.2909754835578484, 0.7090245164421516)
(1.7450526083262474e-05, 0.9999825494739167)
(4.231541269239525e-05, 0.9999576845873076)
(1.460783166651325e-07, 0.9999998539216833)
(0.0003628742774194338, 0.9996371257225806)
(0.9824047082898386, 0.01759529171016135)
(0.9997564865848092, 0.00024351341519076198)
(0.9999836944268992, 1.6305573100842726e-05)
(0.9970229592840892, 0.002977040715910773)
(1.3466800565020698e-05, 0.999986533199435)
(0.9999834479692713, 1.6552030728649067e-05)
(0.9968225327527938, 0.0031774672472062527)
(0.9999942028610157, 5.797138984308302e-06)
(0.9999942028610157, 5.797138984308302e-

(0.9233796578799833, 0.07662034212001666)
(0.8957982759765087, 0.10420172402349123)
(0.9999372729508075, 6.272704919254112e-05)
(0.1802248750366826, 0.8197751249633174)
(0.10900588274388512, 0.8909941172561149)
(0.9990318398177671, 0.0009681601822329142)
(0.003490299490904847, 0.9965097005090952)
(0.9999721583577384, 2.784164226150056e-05)
(0.9431226219623807, 0.05687737803761928)
(0.9946429189724317, 0.005357081027568316)
(0.9997601116578245, 0.00023988834217544635)
(0.9999859660608581, 1.4033939141906462e-05)
(1.9731361524000235e-05, 0.999980268638476)
(0.9999233025247746, 7.669747522543453e-05)
(0.9998823178191031, 0.0001176821808968946)
(0.9999632631128141, 3.673688718589553e-05)
(0.9282650460211843, 0.0717349539788157)
(0.9821435012945716, 0.017856498705428353)
(0.10790101638352756, 0.8920989836164724)
(1.0307360776007357e-07, 0.9999998969263922)
(0.9513493865460487, 0.04865061345395124)
(0.1443025874135383, 0.8556974125864617)
(0.04340652006957102, 0.956593479930429)
(0.999994202

In [50]:
data['prob']=list_good_sent
data.to_csv("./csv/result_sentimental/testing_sentimental_naivebayes.csv",index=False)